In [93]:
!pip3 install requests_html
!pip3 install PyPDF2
!pip3 install rouge
!pip3 install openpyxl



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [94]:
from requests_html import HTMLSession
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup
import re
from openpyxl import Workbook

In [95]:
s= HTMLSession()

In [96]:
def clean_string(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text = text.replace('\n', '')
    return text

def convert_to_map(lst):
    map_dict = {item.split(':')[0].strip(): item.split(':')[1].strip() for item in lst}
    return map_dict

In [97]:
headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.57'}
file= open('input.txt','r')
ids= file.readlines()
base_url='https://pubmed.ncbi.nlm.nih.gov/'
total_abstracts={}
for id in ids:
    pmcid= id.strip()
    r=s.get(base_url+pmcid+'/',headers=headers,timeout=5)
    soup = BeautifulSoup(r.content, 'html.parser')
    div_abstract = soup.find('div', {'class': 'abstract'})

    if div_abstract:
        abstract_components= div_abstract.find_all('p')
        abstract_components=[clean_string(abstract.text.strip()) for abstract in abstract_components]
        total_abstracts[id]=abstract_components

In [ ]:
def write_nested_map_to_excel(nested_map, filename):
    workbook = Workbook()
    sheet = workbook.active
    row = 1
    for outer_key, value_list in nested_map.items():
        sheet.cell(row=row, column=1, value=outer_key)
        # Check if value_list is a list and is not empty before accessing its first item
        if isinstance(value_list, list) and value_list:
            sheet.cell(row=row, column=2, value=value_list[0])
        else:
            sheet.cell(row=row, column=2, value="No data")  # You can change "No data" to any placeholder value you prefer
        row += 1 
    workbook.save(filename)

In [ ]:
write_nested_map_to_excel(total_abstracts, 'output.xlsx')